In [27]:
from transformers import T5ForConditionalGeneration, AutoModel,AutoTokenizer,AutoModelForMaskedLM , Trainer,TrainingArguments,\
BitsAndBytesConfig,pipeline,default_data_collator,DataCollatorWithPadding,DataCollatorForLanguageModeling
from peft import *
import datasets
import torchmetrics
import torch
from hqq.engine.hf import HQQModelForCausalLM
from hqq.models.hf.base import AutoHQQHFModel
from huggingface_hub import snapshot_download
#import deepspeed
import os
import pandas as pd
import json
from var_dump import var_dump
from bs4 import BeautifulSoup

cache_dir='/var/tmp/' #'/proj/ciptmp/ix05ogym/.cache/'
output_dir = cache_dir+'outputs/'


In [32]:
model_name = 'google-t5/t5-small'
tokenizer = AutoTokenizer.from_pretrained(model_name,cache_dir=cache_dir)
model = T5ForConditionalGeneration.from_pretrained(model_name,cache_dir=cache_dir)
model.eval()

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [35]:
embeding_tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-base-en-v1.5',cache_dir =  cache_dir)
embeding_model = AutoModel.from_pretrained('BAAI/bge-base-en-v1.5',cache_dir =  cache_dir)
embeding_model.eval()

print(embeding_model)
print(embeding_model.config)


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [56]:
tokenized_content = embeding_tokenizer(["what is python?","python is a programming language","today is a good day" ,"i love dog!"],padding=True,return_tensors='pt')
#tokenized_query = embeding_tokenizer([],return_tensors='pt')
print(tokenized_content)

#https://chatgpt.com/c/ff4da147-6602-44ad-99e8-58451fe39341
with torch.no_grad():
    o = embeding_model(**tokenized_content).last_hidden_state
    o = o.mean(1)
    #o = model.lm_head(o.last_hidden_state)
    print(o.shape)
    
sentence_embeddings = torch.nn.functional.normalize(o, p=2, dim=1)
sim_score= torch.cosine_similarity(o[0,:],o[1:,:])
print(sim_score)



{'input_ids': tensor([[  101,  2054,  2003, 18750,  1029,   102,     0],
        [  101, 18750,  2003,  1037,  4730,  2653,   102],
        [  101,  2651,  2003,  1037,  2204,  2154,   102],
        [  101,  1045,  2293,  3899,   999,   102,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0]])}
torch.Size([4, 768])
tensor([0.8495, 0.4455, 0.4820])


In [62]:
#https://github.com/FullStackRetrieval-com/RetrievalTutorials/blob/main/tutorials/LevelsOfTextSplitting/5_Levels_Of_Text_Splitting.ipynb

import langchain_experimental
import langchain_experimental.text_splitter 
spliter = langchain_experimental.text_splitter.SemanticChunker(embeding_model,number_of_chunks=2)
spliter.split_text("python is a programming language")

['python is a programming language']